# PRÁCTICA 2 Parte II y III

Javier García Serrano, Ana Gil Molina

## Librerías necesarias

In [1]:
import os
import json
import praw
import re

import torch
import numpy as np
import textwrap

import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score

from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split

from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import TrainingArguments, Trainer

from datasets import Dataset

In [2]:
# Desactivamos esta herramienta de monitorización que viene configurada por defecto en transformers
os.environ['WANDB_DISABLED'] = 'True'

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
# Ahora vamos a ver si nuestro entorno tiene GPU o no
device = torch.device ("cuda:0" if torch.cuda.is_available () else "cpu")
print(device)

cuda:0


## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>6) Question answering usando instructed fine-tuning (calificación 2)</strong></div>

#### Preparación de los datos

En este ejercicio se pide usar instructed fine-tuning para entrenar un modelo capaz de responder preguntas sobre un tema escogido. Para ello, primero es necesario tener un dataset de preguntas, el cual vamos a extraer de Reddit. Hemos escogido el subreddit **r/askscience**, donde se pueden encontrar hilos que hacen preguntas sobre ciencia, y seleccionamos como respuestas válidas los comentarios con mayor puntuación. Reutilizamos el código del ejercicio 1.

In [6]:
# Ruta base para guardar los archivos
ruta_base = os.path.join(os.getcwd(), 'Question Threads')

# Crear la carpeta si no existe
if not os.path.exists(ruta_base):
    os.makedirs(ruta_base)

# Credenciales de la API de Reddit
reddit = praw.Reddit(client_id='Xn-b5v-4Xof1H5CsGCOF_g',
                     client_secret='rvY41h6nkC53avQB2blMmGexMInyJQ',
                     user_agent='script:DLpPLN-Murcia-2025-AyJ (by u/malatuni99)')

# Subreddits de donde se obtendrán los hilos
subreddit_name = 'askscience'

# Función para limpiar nombres de archivos
def limpiar_nombre_archivo(nombre):
    return re.sub(r'[<>:"/\\|?*]', '_', nombre)[:50]  # Reemplaza caracteres no válidos y limita longitud

A la hora de definir la función `get_threads` para obtener los hilos y, de este modo, construir el dataset, tomamos como referencia el formato del dataset ALPACA. Este dataset consta de una columna llamada `instructions`, que contiene las instrucciones sobre lo que debe hacer el modelo (en nuestro caso, la pregunta); una columna `input`, que incluye información adicional que se le puede proporcionar al modelo (en este caso, el texto asociado al hilo); y por último, una columna `output`, que contiene una respuesta válida de referencia (en nuestro caso, los comentarios del hilo con mayor puntuación).

In [7]:
def get_threads(subreddit_name, limit, max_comments):
    subreddit = reddit.subreddit(subreddit_name)
    posts = subreddit.top(limit=limit)
    results = []

    # Moderadores a excluir
    excluded_moderators = ['MockDeath', 'AskScienceModerator']

    for post in posts:
        # Excluir los hilos creados por los moderadores
        if post.author and post.author.name in excluded_moderators:
            continue  # Saltar este hilo y no agregarlo a los resultados

        post.comments.replace_more(limit=0)
        comments = post.comments.list()

        # Seleccionar los comentarios con más votos
        top_comments = sorted(comments, key=lambda x: x.score, reverse=True)[:max_comments]

        # Extraer el texto de los comentarios válidos
        responses = [comment.body for comment in top_comments if comment.body]

        # Agregar el hilo al dataset creando instancias separadas por cada respuesta
        for response in responses:
            results.append({
                'instruction': post.title,
                'input': post.selftext if post.selftext else None,
                'output': response
            })

    return results

Vamos a compilar un dataset de 500 preguntas, y para cada una, tomamos los 3 comentarios con más votos. De esta forma, el dataset tendrá unas 1500 instancias (excluyendo aquellos hilos escritos por moderadores, dado que no suelen ser preguntas).

In [8]:
# Descargar y guardar los hilos
threads = get_threads(subreddit_name, limit=500, max_comments=3)

for idx, post in enumerate(threads, start=1):
    safe_title = limpiar_nombre_archivo(post['instruction'])  # Limpiar el título para usarlo como nombre de archivo
    archivo_path = os.path.join(ruta_base, f'{safe_title}_thread_{idx}.json')

    with open(archivo_path, 'w', encoding='utf-8') as f:
        json.dump(post, f, indent=4, ensure_ascii=False)

# Mostrar el número total de instancias
print(f"Se han cargado un total de {len(threads)} instancias.")

Se han cargado un total de 1440 instancias.


Ahora cargamos los datos almacenados en la carpeta "Question Threads".

In [5]:
# Ruta de la carpeta base 'Question Threads'
script_dir = os.getcwd()                                    # Ruta actual del notebook
threads_dir = os.path.join(script_dir, 'Question Threads')  # Ruta a la carpeta 'Question Threads'

# Listar todos los archivos JSON en la carpeta 'Question Threads'
json_files = [f for f in os.listdir(threads_dir)]

print("Archivos JSON encontrados:", json_files)

Archivos JSON encontrados: ['If you placed wood in a very hot environment with _thread_189.json', 'What does catnip actually do to cats__thread_388.json', 'How come the majority of people in the world are r_thread_1213.json', "Why can't we just inject a ton of power into a pho_thread_242.json", 'Why do certain antidepressants stop you having an _thread_611.json', 'In 1899 _Mile-a-Minute_ Charles Murphy set a bicyc_thread_781.json', 'Is the average human being closer to the size of t_thread_584.json', 'In light of the recent growth of sightings of Tasm_thread_1286.json', 'During a nuclear disaster, is it possible to incre_thread_1189.json', 'Since DNA degrades as we age, would a clone made w_thread_639.json', 'Why exactly are overweight people at higher risk w_thread_1342.json', 'Can you determine the cause of a headache from the_thread_761.json', 'Why do computers and game consoles need to restart_thread_70.json', 'Did the 1918 pandemic have asymptomatic carriers a_thread_459.json', 'A

In [6]:
# Lista para almacenar los datos cargados
dataset = []

# Leer cada archivo JSON y cargar sus datos
for json_file in json_files:
    file_path = os.path.join(threads_dir, json_file)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        dataset.append(data)

# Mostrar la cantidad de instancias cargadas y un ejemplo
print(f"Se han cargado {len(dataset)} instancias.\n")
print("Ejemplo de una instancia:", json.dumps(dataset[0], indent=4, ensure_ascii=False))

Se han cargado 1440 instancias.

Ejemplo de una instancia: {
    "instruction": "If you placed wood in a very hot environment with no oxygen, would it be possible to melt wood?",
    "input": null,
    "output": "And methanol, aka wood alcohol. I believe the technique is called dry distillation. The methanol and other vapors escape the wood and what’s left behind is charcoal. \n\n[Dry distillation of wood](https://yadda.icm.edu.pl/baztech/element/bwmeta1.element.baztech-article-BPP2-0015-0004/c/Lewandowski_GB.pdf)"
}


Dado que debemos reentrenar el modelo, y luego evaluarlo, necesitamos un conjunto de entrenamiento, uno de validación y otro de prueba. Por ello, dividimos el dataset en estos tres conjuntos, tomando una proporción del $80\%$ para entrenamiento, $10\%$ para validación y $10\%$ para prueba.

In [7]:
# Dividir el dataset en entrenamiento (80%), validación (10%) y prueba (10%)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
val_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5, random_state=42)

print(f"Entrenamiento: {len(train_dataset)} instancias")
print(f"Validación: {len(val_dataset)} instancias")
print(f"Prueba: {len(test_dataset)} instancias")

Entrenamiento: 1152 instancias
Validación: 144 instancias
Prueba: 144 instancias


Además, convertimos estos tres subconjuntos de datos en Datasets de Hugging Face, para poder trabajar con ellos de forma más eficiente.

In [8]:
# Convertir los subconjuntos de datos en objetos Dataset de Hugging Face
train_dataset_hf = Dataset.from_list(train_dataset)
val_dataset_hf = Dataset.from_list(val_dataset)
test_dataset_hf = Dataset.from_list(test_dataset)

#### Entrenamiento del modelo Flan T5 Base

Ahora que ya tenemos el dataset, pasamos a cargar el modelo. En este caso, vamos a usar el modelo **Flan T5 Base**. Este modelo es una versión optimizada y mejorada de T5, un modelo de procesamiento del lenguaje natural que sigue un enfoque de *text-to-text*. En particular, Flan T5 Base fue entrenado aplicando *instructed fine-tuning*. Esto permite que se le puedan proporcionar instrucciones acerca de la tarea que debe realizar, mejorando así su capacidad de entender y de responder correctamente, generando respuestas más relevantes para la tarea en cuestión.

In [30]:
# Cargar el modelo y el tokenizador de Flan T5 Base
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model = model.to(device)

Con todo esto, ahora definimos la función `preprocess_data`, encargada de preprocesar los datos. Mediante el `tokenizer` del modelo que acabamos de cargar, se convierten los textos en tokens para que el modelo pueda procesarlos. Por un lado, se concatenan la pregunta y el texto asociado al hilo, y se tokenizan. Por otro lado, se tokenizan las respuestas, y una vez tokenizadas se añaden como etiquetas indicando lo que el modelo debe predecir. Los textos tokenizados tendrán una longitud máxima de 200 tokens, para evitar un elevado coste computacional.

Los textos de entrada que recibirá el modelo comenzarán con una instrucción acerca de lo que se espera que haga, en este caso, que responda a la pregunta proporcionada de forma detallada y con una explicación científica. A continuación, se proporcionará la pregunta en cuestión, y por último se añadirá el texto del hilo correspondiente, para de esta forma proporcionar más contexto sobre la pregunta.

In [60]:
# Función de preprocesamiento
def preprocess_data(batch):
    # Tokenizar las instrucciones y las respuestas
    instructions = batch['instruction']
    input_texts = batch['input']
    output_texts = batch['output']

    # Los valores de input_texts que sean None se reemplazan por una cadena vacía
    input_texts = [input_text if input_text is not None else '' for input_text in input_texts]

    # Concatenar la instrucción "Answer the next question in detail with scientific explanations:" con la pregunta y el texto del hilo
    inputs = ["Answer the next question in detail with scientific explanations: " + instruction + " " + input_text for instruction, input_text in zip(instructions, input_texts)]

    # Tokenizar los textos de entrada y las respuestas
    model_inputs = tokenizer(inputs, max_length=200, truncation=True, padding="max_length", return_tensors="pt").to(device)
    labels = tokenizer(output_texts, max_length=200, truncation=True, padding="max_length", return_tensors="pt").to(device)

    # Añadir los tokens de las respuestas como etiquetas
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Ahora, gracias a la función `preprocess_data` que acabamos de definir, podemos tokenizar los diferentes subconjuntos de nuestro dataset. Mediante `map` podemos aplicar dicha función al conjunto de entrenamiento, validación y prueba por batches, haciendo el proceso más eficiente.

In [61]:
# Aplicar el preprocesamiento para todos los subconjuntos de datos
train_dataset_flant5 = train_dataset_hf.map(preprocess_data, batched=True)
val_dataset_flant5 = val_dataset_hf.map(preprocess_data, batched=True)
test_dataset_flant5 = test_dataset_hf.map(preprocess_data, batched=True)

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

Ahora definimos los parámetros para el entrenamiento, usando `TrainingArguments`, donde se definen, entre otros, los siguientes parámetros:

- `output_dir`: carpeta para guardar los resultados.

- `eval_strategy`: se evalúa el modelo al final de cada época.

- `learning_rate`: tasa de aprendizaje.

- `batch_size`: tamaño del lote para entrenamiento y para validación.

- `num_train_epochs`: número de épocas, esto es, de pasadas completas por el conjunto de entrenamiento.

- `weight_decay`: regularización para evitar sobreajuste.

In [62]:
# Asegurarse de que existen las rutas necesarias
os.makedirs(os.path.join(os.getcwd(), 'results', 'results_FlanT5_Base'), exist_ok=True)
os.makedirs(os.path.join(os.getcwd(), 'logs', 'logs_FlanT5_Base'), exist_ok=True)

# Configurar argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir = os.path.join(os.getcwd(), 'results', 'results_FlanT5_Base'),
    eval_strategy = "epoch",
    learning_rate = 5e-5,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    weight_decay = 0.01,
    save_total_limit = 2,
    save_steps = 100,  # Guardar el modelo entrenado cada 100 pasos
    logging_dir = os.path.join(os.getcwd(), 'logs', 'logs_FlanT5_Base'),
    logging_steps = 10,
    push_to_hub = False
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


A continuación, se configura la clase `Trainer`, la cual facilita el entrenamiento y la evaluación del modelo. Se le pasan el modelo, los argumentos de entrenamiento, los datasets de entrenamiento y validación, y el tokenizador.

In [63]:
# Vaciar caché de GPU
torch.cuda.empty_cache()

# Configurar Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset_flant5,
    eval_dataset = val_dataset_flant5,
    tokenizer = tokenizer
)

<ipython-input-63-4e8d3b6a345f>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Finalmente, entrenamos y guardamos el modelo.

In [64]:
# Vaciar caché de GPU
torch.cuda.empty_cache()

# Entrenar el modelo
trainer.train()

# Guardar el modelo
os.makedirs(os.path.join(os.getcwd(), 'models', 'model_FlanT5_Base'), exist_ok=True)
trainer.save_model(os.path.join(os.getcwd(), 'models', 'model_FlanT5_Base'))

Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan


#### Evaluación del modelo

Una vez realizado el instructed fine-tuning del modelo Flan T5 Base, pasamos a evaluar dicho modelo en el conjunto de prueba que habíamos guardado anteriormente. Comenzamos cargando el modelo y el tokenizador.

In [ ]:
# Ruta del modelo guardado
model_path = os.path.join(os.getcwd(), 'models', 'model_FlanT5_Base')

# Cargar el modelo
model = T5ForConditionalGeneration.from_pretrained(model_path, device_map="auto")

# Cargar el tokenizador
tokenizer = T5Tokenizer.from_pretrained(model_path)

Con el modelo ya cargado, pasamos a realizar las predicciones. El parámetro `max_new_tokens` controla el número máximo de tokens que el modelo generará para la salida. Dado que nuestros datos provienen del subreddit r/askscience, donde las respuestas suelen ser complejas y detalladas, tomamos un valor de $200$ para que el modelo pueda proporcionar respuestas largas y bien estructuradas, pero sin ser excesivamente largas.

In [65]:
# Mover el modelo al dispositivo disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Realizar las predicciones
model.eval()  # Establecer el modelo en modo evaluación
predictions = []
batch_size = 8

# Realizar las predicciones por batches
for i in range(0, len(test_dataset_flant5), batch_size):
    batch = test_dataset_flant5.select(range(i, min(i+batch_size, len(test_dataset_flant5))))  # Obtener un lote del dataset

    # Obtener los input_ids y attention_mask del lote
    input_ids = torch.tensor([example['input_ids'] for example in batch]).to(device)
    attention_mask = torch.tensor([example['attention_mask'] for example in batch]).to(device)

    # Realizar la predicción
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=200)

    # Decodificar las predicciones
    for output in outputs:
        predicted_text = tokenizer.decode(output, skip_special_tokens=True)

        # Almacenar la predicción
        predictions.append(predicted_text)

Para evaluar los resultados, comenzaremos analizando algunos ejemplos de manera manual. Para ello, seleccionaremos varios hilos del conjunto de test, y mostraremos la pregunta correspondiente junto con la respuesta de referencia. A continuación, mostraremos la respuesta generada por el modelo y evaluaremos si esta tiene sentido y es coherente.

In [67]:
# Configurar el ancho máximo de cada línea
max_width = 140

print('-Ejemplo 1-')
print('Pregunta:')
print(textwrap.fill(test_dataset[0]['instruction'], width=max_width))

print('\nRespuesta de referencia:')
print(textwrap.fill(test_dataset[0]['output'], width=max_width))

print('\nRespuesta generada por el modelo:')
print(textwrap.fill(predictions[0], width=max_width))

-Ejemplo 1-
Pregunta:
Theoretically, if the whole world isolates itself for a month, could the flu, it's various strains, and future mutated strains be a thing of
the past? Like, can we kill two birds with one stone?

Respuesta de referencia:
Influenza, along with many other viruses, such as coronaviruses, have animal reservoirs of disease that the virus exists within. For
influenza this is the bird population.   These reservoirs are a major focus of investigation for the medical community, as they provide a
point of reinfection for the human population, even if we were to eliminate the circulating virus in our own population.
https://academic.oup.com/jid/article/216/suppl_4/S493/4162042  Some infections, such as measles and polio could theoretically eliminated by
isolation, but vaccines are proving to be a more effective mechanism for their elimination.

Respuesta generada por el modelo:
The flu virus is a viral infection that causes a viral infection. The virus can be transmitted by 

Para este ejemplo, observamos que la respuesta proporcionada por el modelo, si bien está relacionada con la temática de la pregunta, tiene claras deficiencias. En primer lugar, la pregunta busca un análisis sobre si el aislamiento global podría erradicar el virus de la gripe, pero la respuesta no responde a esta pregunta. Además, entra en un bucle en el que repite una y otra vez la misma frase.

In [68]:
print('-Ejemplo 2-')
print('Pregunta:')
print(textwrap.fill(test_dataset[1]['instruction'], width=max_width))

print('\nRespuesta de referencia:')
print(textwrap.fill(test_dataset[1]['output'], width=max_width))

print('\nRespuesta generada por el modelo:')
print(textwrap.fill(predictions[1], width=max_width))

-Ejemplo 2-
Pregunta:
If a heart is a muscle, why doesn’t it ever get tired of beating but things like my arms and legs do?

Respuesta de referencia:
Correct. Lactate is a fuel that's used by the heart, brain and even Type 1 muscle fibers both during rest and exercise. In fact, there's
excellent evidence to suggest that the preferred pathway to burn glucose is through the lactate rather than the pyruvate pathway. Here's an
excellent review.  https://www.cell.com/cell-metabolism/comments/S1550-4131(18)30186-4

Respuesta generada por el modelo:
The heart is a muscle that is able to pump blood and oxygen through the circulatory system.


En este segundo ejemplo, la respuesta generada por el modelo es técnicamente correcta, aunque demasiado breve e incompleta. No responde a la pregunta principal, la cual solicita una explicación de por qué el corazón no se cansa de bombear sangre. Además, tampoco se ajusta al estilo esperado de una respuesta típica de Reddit, que podría incluir una explicación más detallada, y que en algunos casos podría presentar un toque de humor para hacerla más accesible.

In [70]:
print('-Ejemplo 3-')
print('Pregunta:')
print(textwrap.fill(test_dataset[2]['instruction'], width=max_width))

print('\nRespuesta de referencia:')
print(textwrap.fill(test_dataset[2]['output'], width=max_width))

print('\nRespuesta generada por el modelo:')
print(textwrap.fill(predictions[2], width=max_width))

-Ejemplo 3-
Pregunta:
In the U.S., if the polio vaccination rate was the same as COVID-19, would we still have polio?

Respuesta de referencia:
That's the answer, yeah. Kids ended up in iron lungs for the rest of their lives. Reality is, that moves a lot more people than when people
on the other end of the age spectrum are dying.

Respuesta generada por el modelo:
COVID-19 is a disease that causes a polio virus to enter the body. The polio virus is transmitted by a virus called a poliovirus. The polio
virus is transmitted by a virus called a poliovirus. The polio virus is transmitted by a virus called a poliovirus. The polio virus is
transmitted by a virus called a poliovirus. The polio virus is transmitted by a virus called a poliovirus. The polio virus is transmitted by
a virus called a poliovirus. The polio virus is transmitted by a virus called a poliovirus. The polio virus is transmitted by a virus called
a poliovirus. The polio virus is transmitted by a virus called a poliovirus.

En este tercer ejemplo, de nuevo la respuesta es inadecuada y presenta una serie de problemas. Por ejemplo, afirma que COVID-19 es una enfermedad que causa que el virus polio entre al cuerpo, lo cual no es correcto. Además, la respuesta entra en un bucle repetitivo sin sentido. Por último, no se responde a la pregunta, la cual busca conocer si en caso de que la tasa de vacunación del polio fuera igual al del COVID-19, se podría erradicar esta enfermedad.

La deficiencia del modelo puede deberse a diversas causas. En primer lugar, el modelo Flan T5 Base no tiene una gran complejidad en comparación con el modelo Gemma (2B), el cual se recomendaba usar en este ejercicio. Decidimos usar Flan T5 Base debido a limitaciones de memoria con Gemma (2B), pero esto claramente ha tenido como resultado un desempeño peor.

Por otro lado, el tipo de datos escogidos para hacer fine-tuning y evaluar el modelo, extraídos de un subreddit sobre preguntas científicas, podrían ser demasiado complejas para un modelo como Flan T5 Base, que podría haber sido entrenado inicialmente con datos posiblemente más generales y menos específicos al dominio científico. Además, este dataset extraído de Reddit podría contener preguntas y respuestas demasiado centradas en contextos concretos, que podrían ser difíciles de entender para un modelo como Flan T5 Base.

Modelos más grandes como Gemma (2B) podrían tener una mayor capacidad para manejar la complejidad y diversidad en los datos científicos, lo que podría haber resultado en un mejor rendimiento en este caso.

Para evaluar el modelo de manera más objetiva, ahora vamos a utilizar diferentes métricas para cuantificar la calidad de las respuestas generadas. Algunas métricas comunes en tareas de generación de texto que podemos usar para evaluar el modelo son:

1. ROUGE-N: Esta métrica compara la superposición de n-gramas (secuencias de n palabras consecutivas) entre las predicciones generadas por el modelo y las respuestas de referencia. Por ejemplo, con ROUGE-1 se comparan unigramas, esto es, palabras individuales, mientras que con ROUGE-2 se comparan bigramas, es decir, pares de palabras consecutivas. Además, ROUGE-L evalúa la longitud de la subsecuencia común más larga entre la respuesta generada y la respuesta de referencia.

2. METEOR: Esta métrica evalúa la similitud entre las respuestas generadas y las de referencia tomando en cuenta coincidencias exactas de palabras, sinónimos, raíces y el orden de las palabras. Además, prioriza el equilibrio entre precisión y recall.

Comencemos con ROUGE-N. A continuación, se define una función que permite calcular la métrica ROUGE-N para evaluar la calidad de las respuestas generadas en comparación con las respuestas de referencia. Utiliza el paquete `rouge_scorer` para obtener las métricas ROUGE-1, ROUGE-2 y ROUGE-L. Para ello, recorre las listas de predicciones y referencias, y para cada par, va calculando las métricas. Finalmente, se calcula el promedio de cada métrica y se devuelve como un diccionario.

In [71]:
# Función para calcular ROUGE-N
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}

    # Recorrer todas las predicciones y referencias
    for pred, ref in zip(predictions, references):
        score = scorer.score(ref, pred)
        for key in scores:
            scores[key].append(score[key].fmeasure)

    # Promediar las puntuaciones ROUGE
    avg_scores = {key: sum(value) / len(value) for key, value in scores.items()}
    return avg_scores

In [72]:
# Obtener las respuestas de referencia del dataset
test_references = [test_example['output'] for test_example in test_dataset]

In [73]:
# Evaluar ROUGE-N
rouge_scores = compute_rouge(predictions, test_references)

print("ROUGE-1:", rouge_scores["rouge1"])
print("ROUGE-2:", rouge_scores["rouge2"])
print("ROUGE-L:", rouge_scores["rougeL"])

ROUGE-1: 0.07804530472064046
ROUGE-2: 0.0077273511143935575
ROUGE-L: 0.05909889247265389


De nuevo, vemos un desempeño deficiente al observar las métricas ROUGE. Por ejemplo, un valor de $0.078$ indica una coincidencia muy baja en las palabras individuales. Esto sugiere que las respuestas generadas contienen pocas palabras relevantes o no coinciden bien con el vocabulario de las referencias. Por otro lado, el valor de ROUGE-2 es extremadamente bajo, lo que muestra que las combinaciones de pares de palabras no se alinean entre las respuestas generadas y las referencias. Por último, un valor de $0.059$ indica que las respuestas generadas tienen muy poca similitud con las referencias en cuanto a la estructura esperada de las oraciones.

Ahora, vamos a calcular la métrica METEOR sobre nuestros datos de test. Para ello, definimos una función `compute_meteor`, que recorre las listas de predicciones y referencias, y para cada par, calcula la métrica METEOR. Finalmente, se calcula el promedio de la métrica y se devuelve su valor.

In [90]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [91]:
# Función para calcular METEOR
def compute_meteor(predictions, references):
    scores = []

    for pred, ref in zip(predictions, references):
        # Tokenizar las predicciones y las referencias
        pred_tokens = word_tokenize(pred)
        ref_tokens = word_tokenize(ref)

        # Calcular METEOR para cada par de predicción y referencia
        score = meteor_score([ref_tokens], pred_tokens)
        scores.append(score)

    # Promediar las puntuaciones METEOR
    avg_score = sum(scores) / len(scores) if scores else 0
    return avg_score

In [92]:
# Evaluar METEOR para las predicciones
meteor_score_value = compute_meteor(predictions, test_references)

# Imprimir la puntuación METEOR
print(f"METEOR: {meteor_score_value:.4f}")

METEOR: 0.0419


Finalmente, el METEOR score de $0.0419$ es bastante bajo, lo que refuerza la idea de que el modelo no está generando respuestas que coincidan bien con las referencias en términos de contenido, estructura, o semántica. 